# Import Packages

In [1]:
import keras
import os
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Lambda
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from pickle import dump, load
import re

Using TensorFlow backend.


In [2]:
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

# Import Dataset and Encode

In [4]:
raw_text = load_doc("data/shakespeare.txt")
raw_text = "".join(filter(lambda x: not x.isdigit(), raw_text)) 
raw_text = raw_text.lower().strip()
raw_text = re.sub(r'(\n\s*)+\n', '\n\n', raw_text)

chars = sorted(list(set(raw_text)))
mapping = dict((c, i) for i, c in enumerate(chars))
    
# organize into sequences of characters
length = 40
step = 1
sequences = []
for i in range(length, len(raw_text), step):
    seq = raw_text[i-length:i+1]
    sequences.append(seq)
    
print('Total Sequences: %d' % len(sequences))

encoded_sequences = []

for s in sequences:
    encoded_seq = [mapping[char] for char in s]
    encoded_sequences.append(encoded_seq)

es = np.array(encoded_sequences)
X, y = es[:,:-1], es[:,-1]
es = [to_categorical(x, num_classes=len(chars)) for x in X]
X = np.array(es)
y = to_categorical(y, num_classes=len(chars))

Total Sequences: 94402


# Define RNN-LSTM and Train

In [5]:
# define model
model = Sequential()
model.add(LSTM(200, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(len(chars), activation='softmax'))
print(model.summary())

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, epochs=100, verbose=2)
 
# save the model to file
model.save('rnn_model/lstm_shakespeare.h5')
# save the mapping
dump(mapping, open('rnn_model/shakespeare_mapping.pkl', 'wb'))

W0315 19:36:05.221344 4567434688 deprecation_wrapper.py:119] From /Users/jma/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0315 19:36:05.233913 4567434688 deprecation_wrapper.py:119] From /Users/jma/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0315 19:36:05.235983 4567434688 deprecation_wrapper.py:119] From /Users/jma/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0315 19:36:05.408257 4567434688 deprecation_wrapper.py:119] From /Users/jma/anaconda3/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 200)               191200    
_________________________________________________________________
dense_1 (Dense)              (None, 38)                7638      
Total params: 198,838
Trainable params: 198,838
Non-trainable params: 0
_________________________________________________________________
None


W0315 19:36:05.424273 4567434688 deprecation_wrapper.py:119] From /Users/jma/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0315 19:36:05.608199 4567434688 deprecation.py:323] From /Users/jma/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0315 19:36:06.040989 4567434688 deprecation_wrapper.py:119] From /Users/jma/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/100
 - 88s - loss: 2.2585 - acc: 0.3449
Epoch 2/100
 - 83s - loss: 1.8342 - acc: 0.4501
Epoch 3/100
 - 84s - loss: 1.6844 - acc: 0.4874
Epoch 4/100
 - 83s - loss: 1.5895 - acc: 0.5119
Epoch 5/100
 - 81s - loss: 1.5125 - acc: 0.5320
Epoch 6/100
 - 81s - loss: 1.4471 - acc: 0.5501
Epoch 7/100
 - 81s - loss: 1.3865 - acc: 0.5663
Epoch 8/100
 - 81s - loss: 1.3264 - acc: 0.5837
Epoch 9/100
 - 81s - loss: 1.2698 - acc: 0.6000
Epoch 10/100
 - 92s - loss: 1.2168 - acc: 0.6154
Epoch 11/100
 - 89s - loss: 1.1638 - acc: 0.6308
Epoch 12/100
 - 91s - loss: 1.1166 - acc: 0.6454
Epoch 13/100
 - 93s - loss: 1.0702 - acc: 0.6597
Epoch 14/100
 - 92s - loss: 1.0313 - acc: 0.6718
Epoch 15/100
 - 95s - loss: 0.9939 - acc: 0.6831
Epoch 16/100
 - 95s - loss: 0.9616 - acc: 0.6922
Epoch 17/100
 - 90s - loss: 0.9306 - acc: 0.7018
Epoch 18/100
 - 91s - loss: 0.9007 - acc: 0.7114
Epoch 19/100
 - 93s - loss: 0.8783 - acc: 0.7185
Epoch 20/100
 - 89s - loss: 0.8555 - acc: 0.7249
Epoch 21/100
 - 90s - loss: 0

In [9]:
# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # one hot encode
        encoded = to_categorical(encoded, num_classes=len(mapping))
        encoded = encoded.reshape(1, encoded.shape[1], encoded.shape[2])
        # predict character
        yhat = model.predict_classes(encoded, verbose=0)
        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        # append to input
        in_text += char
    return in_text


In [ ]:
model = load_model('rnn_model/lstm_shakespeare.h5')
mapping = load(open('rnn_model/shakespeare_mapping.pkl', 'rb'))
temp_weights = [layer.get_weights() for layer in model.layers]

for temp in [1.5, 0.75, 0.25]:
    print("RNN Sonnet Generation at Temperature =", temp)
    print("=========================================================")
    
    model = Sequential()
    model.add(LSTM(200, input_shape=(X.shape[1], X.shape[2])))
    model.add(Lambda(lambda x: x / temp))
    model.add(Dense(len(chars), activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.layers[0].set_weights(temp_weights[0])
    model.layers[2].set_weights(temp_weights[1])

    print(generate_seq(model, mapping, 40, "shall i compare thee to a summer's day?\n", 1200))
    print("=========================================================")


RNN Sonnet Generation at Temperature = 1.5
shall i compare thee to a summer's day?
thou art to me than my borning old me;
nor blied of such time's life hath pleas,
to make me give then my thoughts more doth live,
that i am seem long secalled from love
after my dead see thee i agains my heart:
another till in his midder from their rest,
being franterfur vice with too much deeper,
  in sleep and thine ow erriming lies,
the canker blooms for woods i loved his own,
for no false eleminared of when i praise.

without thy changing chy complexion dind.
for whose will in these all the rearors.
for if thou wilt, if ever sye can let me,
which in the bressed with mire transe so die!
  but what's so breast-to me, i see the world i sin,
that in my brow, in wind, nor ston and bring forthing most,
but by soir birds with thee, and they so lie:
then can i draw a bearing of their bight,
i see deaths infleirar end your advise.

why so large of many a houry of love,
they are but dick-latch she lies of thee